In [14]:
import pandas as pd, urllib, os, re, matplotlib.pyplot as plt, numpy as np, dateparser, codecs,geopandas, shapely, folium, json, ftplib, networkx as nx, community,statsmodels.formula.api  as smf, progressbar
from sklearn.preprocessing import StandardScaler 
from bs4 import BeautifulSoup as BS
from itertools import chain
%matplotlib inline

def int2str_id(ID, l = 5, frontpad = True, pad = '0'):
    if type(ID)==str:
        return ID
    x = str(int(ID))    
    if frontpad:
        return pad*(l-len(x))+x
    else:
        return x+pad*(l-len(x))

In [106]:
os.listdir('D://safegraph/')

['coreplaces',
 'core_poi',
 'monthly_backfill',
 'monthly_places',
 'open_census',
 'safegraph_open_census_data.tar.gz',
 'social_distancing',
 'transactions',
 'weekly_patterns',
 'zz_documentation']

# Take a look at each dataset

## Social Distancing

In [26]:
sdpath = 'D://safegraph/social_distancing/'
os.listdir(sdpath+'2020/01/01')
dts = list(pd.date_range('1/1/2020', '5/15/2020/'))
subdirs = ['{}/{}/{}/'.format(c.year, int2str_id(c.month, l=2), int2str_id(c.day, l =2)) for c in dts]
sdflist = list(chain(*[[sdpath + d + f for f in os.listdir(sdpath + d)] for d in subdirs]))

In [27]:
f = np.random.choice(sdflist)
print(f)

D://safegraph/social_distancing/2020/03/27/2020-03-27-social-distancing.csv.gz


In [28]:
d = pd.read_csv(f, compression = 'gzip')

In [30]:
d.head()

,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home
0,10330210004,2020-03-27T00:00:00-05:00,2020-03-28T00:00:00-05:00,69,9121.0,"{""16001-50000"":8,""0"":21,"">50000"":9,""2001-8000""...","{""16001-50000"":22,"">50000"":86,""<1000"":32,""2001...",19,840,"{""721-1080"":9,""361-720"":4,""61-360"":7,""<60"":19,...","[38,42,44,43,40,41,39,37,35,38,36,36,36,35,34,...",4,1,"{""281419503001"":2,""281419502003"":2,""0103302030...",1,63,163,"{""21-45"":3,""481-540"":2,""721-840"":2,""1201-1320""...",77,"{""26-50"":1,""51-75"":10,""0-25"":19,"">100"":5,""76-1..."
1,10730049022,2020-03-27T00:00:00-05:00,2020-03-28T00:00:00-05:00,151,8264.0,"{""16001-50000"":15,""0"":31,"">50000"":34,""2001-800...","{""16001-50000"":256,"">50000"":169,""<1000"":102,""2...",33,0,"{""721-1080"":4,""361-720"":14,""61-360"":6,""<60"":95...","[31,29,32,29,34,33,34,31,30,29,22,26,28,25,27,...",9,1,"{""011150405022"":1,""011170302131"":1,""0111703021...",1,197,668,"{""21-45"":1,""481-540"":4,""541-600"":1,""46-60"":7,""...",0,"{""26-50"":2,""51-75"":1,""0-25"":94,"">100"":1,""76-10..."
2,11210118001,2020-03-27T00:00:00-05:00,2020-03-28T00:00:00-05:00,55,3822.0,"{""16001-50000"":7,""0"":14,"">50000"":7,""2001-8000""...","{""16001-50000"":25,"">50000"":37,""<1000"":416,""200...",14,619,"{""721-1080"":11,""361-720"":6,""61-360"":9,""<60"":16...","[24,23,27,28,29,27,24,26,24,26,23,18,23,26,20,...",2,1,"{""011150405022"":2,""011210109002"":2,""0108104130...",4,49,221,"{""21-45"":5,""541-600"":3,""46-60"":1,""721-840"":6,""...",63,"{""26-50"":6,""51-75"":7,""0-25"":18,"">100"":1,""76-10..."
3,11250106021,2020-03-27T00:00:00-05:00,2020-03-28T00:00:00-05:00,352,16477.0,"{""16001-50000"":99,""0"":80,"">50000"":44,""2001-800...","{""16001-50000"":38,"">50000"":76,""<1000"":85,""2001...",82,827,"{""721-1080"":55,""361-720"":41,""61-360"":20,""<60"":...","[206,205,212,215,211,207,195,188,178,168,160,1...",27,16,"{""010730102002"":10,""011150405022"":1,""011250106...",17,122,747,"{""21-45"":28,""481-540"":5,""541-600"":8,""46-60"":12...",79,"{""26-50"":22,""51-75"":44,""0-25"":95,"">100"":8,""76-..."
4,21880002003,2020-03-27T00:00:00-08:00,2020-03-28T00:00:00-08:00,21,420.0,"{""1-1000"":9,"">50000"":3,""0"":5}","{"">50000"":464,""<1000"":324}",4,200,"{""721-1080"":3,""361-720"":2,""61-360"":2,""<60"":12,...","[4,3,3,4,4,2,3,3,3,3,3,3,1,6,3,3,1,5,5,2,3,1,3,3]",1,1,"{""021880002003"":11,""021880002001"":2,""021880002...",1,265,35,"{""541-600"":1,""721-840"":1,""301-360"":2,""<20"":6,""...",49,"{""0-25"":8,""76-100"":6,""51-75"":1,""26-50"":7}"


## Weekly Store Patterns

In [38]:
wkpatpath = 'D://safegraph/weekly_patterns/'
os.listdir(wkpatpath)

['home-summary-file', 'main-file', 'normalization-stats', 'release-metadata']

In [42]:
wk_main_flist = os.listdir(wkpatpath + 'main-file/')
f = np.random.choice(wk_main_flist)

In [44]:
d = pd.read_csv(wkpatpath+'main-file/'+f, compression = 'gzip')
d.head()

,safegraph_place_id,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,date_range_start,...,poi_cbg,visitor_home_cbgs,visitor_daytime_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_week_brand,device_type
0,sg:0013bd02b3564a9187a52f8e2799233e,Small Business Transactions,1007 Mill Crescent Pl Ste 274,Arlington,TX,76006,US,NaN,NaN,2019-01-07T00:00:00-06:00,...,4.843911e+11,"{""484391131041"":11,""484391141042"":4,""484391131...","{""484391131041"":9,""484391135113"":4,""4843911155...","{""US"":33}",5405.0,74.0,"{""<5"":2,""5-20"":15,""21-60"":7,""61-240"":13,"">240""...","{""Tom Thumb Food & Pharmacy"":17,""QuikTrip"":9,""...","{""Walmart"":14,""Jack in the Box"":14,""Valero Ene...","{""android"":26,""ios"":7}"
1,sg:001955fa1c994b4c8c877316a66dd986,McDonald's,1134 280 By Pass,Phenix City,AL,36867,US,SG_BRAND_5179b21fc1d50950b99b4eecaa48c614,McDonald's,2019-01-07T00:00:00-06:00,...,1.113031e+10,"{""011130306003"":15,""011130309022"":10,""01113030...","{""011130306003"":13,""011130309021"":13,""01113030...","{""US"":168}",11767.0,10.0,"{""<5"":7,""5-20"":146,""21-60"":29,""61-240"":6,"">240...","{""Dollar General"":14,""Piggly Wiggly"":11,""Shell...","{""Walmart"":27,""Marathon"":23,""Dollar General"":2...","{""android"":88,""ios"":78}"
2,sg:001e39c6b18645a5950b13a278b242c3,Peking House,9617 Liberty Ave,Ozone Park,NY,11417,US,NaN,NaN,2019-01-07T00:00:00-05:00,...,3.608100e+11,"{""360810040021"":7,""360810086001"":5,""3608100200...","{""360810040021"":8,""360610048006"":5,""3600502270...","{""US"":91}",2477.0,21.0,"{""<5"":2,""5-20"":57,""21-60"":22,""61-240"":23,"">240...","{""Subway"":9,""Stop & Shop"":7,""P.C. Richard & So...","{""McDonald's"":14,""Dunkin'"":13,""Popeyes Louisia...","{""android"":53,""ios"":40}"
3,sg:0023804ee36c4056886ef54e5a0cb773,American Express Financial Advisors,4210 Metro Pkwy Ste 230,Fort Myers,FL,33916,US,NaN,NaN,2019-01-07T00:00:00-05:00,...,1.207100e+11,"{""120710102042"":4,""120710401273"":4,""1207104020...","{""120710012011"":5,""120710015024"":4,""1207100120...","{""US"":20}",10517.0,165.0,"{""<5"":1,""5-20"":12,""21-60"":5,""61-240"":13,"">240""...","{""Subway"":99,""Edward Jones"":12,""Citi Trends"":1...","{""7-Eleven"":22,""Sam's Club"":22,""Subway"":20,""Ra...","{""android"":10,""ios"":12}"
4,sg:00267c6356804259b6c92ba31c842f5a,Akimel Smoke Shop 2,190 W Pima St,Sacaton,AZ,85147,US,NaN,NaN,2019-01-07T00:00:00-07:00,...,4.021941e+10,"{""040219412004"":4,""040219413001"":4,""0401311661...","{""040219412004"":8,""040219413001"":4,""0402194130...","{""US"":25}",15018.0,22.5,"{""<5"":0,""5-20"":19,""21-60"":10,""61-240"":7,"">240"":4}","{""QuikTrip"":20,""Simon mall"":20,""76"":17,""Circle...","{""Fry's Food & Drug Stores"":24,""Subway"":17,""76...","{""android"":19,""ios"":4}"


In [45]:
d.shape

(3870885, 25)

In [46]:
d.columns.values

array(['safegraph_place_id', 'location_name', 'street_address', 'city',
       'region', 'postal_code', 'iso_country_code', 'safegraph_brand_ids',
       'brands', 'date_range_start', 'date_range_end', 'raw_visit_counts',
       'raw_visitor_counts', 'visits_by_day', 'visits_by_each_hour',
       'poi_cbg', 'visitor_home_cbgs', 'visitor_daytime_cbgs',
       'visitor_country_of_origin', 'distance_from_home', 'median_dwell',
       'bucketed_dwell_times', 'related_same_day_brand',
       'related_same_week_brand', 'device_type'], dtype=object)

In [47]:
d.describe()

,postal_code,raw_visit_counts,raw_visitor_counts,poi_cbg,distance_from_home,median_dwell
count,3.870885e+06,3.870885e+06,3.870885e+06,3.870861e+06,2.905702e+06,3.870885e+06
mean,5.233478e+04,5.857590e+01,3.787584e+01,2.758613e+11,1.669810e+04,7.734219e+01
std,2.949101e+04,2.658053e+02,1.484261e+02,1.625946e+11,1.097512e+05,1.266949e+02
min,0.000000e+00,1.000000e+00,1.000000e+00,1.001020e+10,1.000000e+00,0.000000e+00
25%,2.906300e+04,7.000000e+00,5.000000e+00,1.209502e+11,4.260000e+03,1.700000e+01
50%,4.906800e+04,2.200000e+01,1.400000e+01,2.705303e+11,7.446000e+03,3.900000e+01
75%,7.861300e+04,5.500000e+01,3.600000e+01,4.200595e+11,1.235000e+04,8.050000e+01
max,9.999900e+04,1.899420e+05,6.062400e+04,7.801097e+11,9.619368e+06,1.440000e+03


In [51]:
d.visits_by_day.head().map(eval).map(sum)

0     59
1    192
2    125
3     54
4     40
Name: visits_by_day, dtype: int64

In [49]:
d.raw_visitor_counts.head()

0     35
1    171
2     93
3     21
4     25
Name: raw_visitor_counts, dtype: int64

In [52]:
d.visits_by_each_hour.head()

0    [1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1...
1    [0,0,0,0,0,0,0,0,0,1,1,1,1,1,4,2,1,0,6,1,1,0,0...
2    [0,0,0,0,0,1,0,0,0,0,1,1,1,0,3,1,1,3,0,2,0,0,2...
3    [0,0,0,0,0,0,0,0,1,2,0,2,1,1,1,0,0,0,1,1,0,0,0...
4    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0...
Name: visits_by_each_hour, dtype: object

In [53]:
d.poi_cbg.head() # census block group

0    4.843911e+11
1    1.113031e+10
2    3.608100e+11
3    1.207100e+11
4    4.021941e+10
Name: poi_cbg, dtype: float64

In [54]:
d.visitor_home_cbgs.head() # visitors census block groups

0    {"484391131041":11,"484391141042":4,"484391131...
1    {"011130306003":15,"011130309022":10,"01113030...
2    {"360810040021":7,"360810086001":5,"3608100200...
3    {"120710102042":4,"120710401273":4,"1207104020...
4    {"040219412004":4,"040219413001":4,"0401311661...
Name: visitor_home_cbgs, dtype: object

In [55]:
d.street_address.head()

0    1007 Mill Crescent Pl Ste 274
1                 1134 280 By Pass
2                 9617 Liberty Ave
3          4210 Metro Pkwy Ste 230
4                    190 W Pima St
Name: street_address, dtype: object

In [59]:
wk_home_summary = os.listdir(wkpatpath + 'home-summary-file/')
f = np.random.choice(wk_home_summary)
f

'2019-07-08-home-panel-summary.csv'

In [60]:
d = pd.read_csv(wkpatpath+'home-summary-file/'+f)

In [61]:
d.head() # Basically just the number of devices at each census block at home during week.

,date_range_start,date_range_end,state,census_block_group,number_devices_residing
0,2019-07-08T00:00:00-08:00,2019-07-15T00:00:00-08:00,ak,21220006005,55
1,2019-07-08T00:00:00-08:00,2019-07-15T00:00:00-08:00,ak,20200010002,114
2,2019-07-08T00:00:00-08:00,2019-07-15T00:00:00-08:00,ak,20200014004,58
3,2019-07-08T00:00:00-08:00,2019-07-15T00:00:00-08:00,ak,20900001001,73
4,2019-07-08T00:00:00-08:00,2019-07-15T00:00:00-08:00,ak,20200029004,34


In [85]:
wk_norm_flist = os.listdir(wkpatpath + 'normalization-stats/')
f = np.random.choice(wk_norm_flist)
f

'2019-02-11-normalization-stats.csv'

In [86]:
d = pd.read_csv(wkpatpath+'/normalization-stats/'+f)
d.head()

,year,month,day,total_visits,total_devices_seen,total_home_visits,total_home_visitors
0,2019,2,11,57743412,18932305,24868952,15131435
1,2019,2,12,61636611,19125594,26140653,15243757
2,2019,2,13,63347787,18444063,25468864,14576636
3,2019,2,14,65247600,18352364,25756984,14435483
4,2019,2,15,63239072,18309891,24589899,14124838


# Transactions

In [62]:
transpath = 'D://safegraph/transactions/'
os.listdir(transpath)

['cut-1-daily-spend-by-zip',
 'cut-2-daily-spend-by-zip-by-mcc',
 'cut-3-daily-spend-by-brand',
 'cut-4-daily-spend-at-walmart',
 'cut-5-daily-federal-deposits-by-zip',
 'monthly-panel-summary']

In [76]:
os.listdir(transpath + 'cut-1-daily-spend-by-zip/'), os.listdir(transpath + 'cut-1-daily-spend-by-zip/2020-04-05/')

(['2020-04-05', '2020-04-22', '2020-05-19'],
 ['cut-1-daily-spend-by-zip-20170101-20200405.csv.gz'])

In [88]:
d = pd.read_csv(transpath + 'cut-1-daily-spend-by-zip/2020-04-05/'  + 'cut-1-daily-spend-by-zip-20170101-20200405.csv.gz', compression = 'gzip')
d.columns = ['date', 'cardholder_zip', 'num_cards', 'num_trans', 'total_spend']
d.head()

,date,cardholder_zip,num_cards,num_trans,total_spend
0,2017-01-01,1566,8,14,570.01
1,2017-01-01,1887,18,29,651.23
2,2017-01-01,1945,8,9,86.10
3,2017-01-01,1969,4,4,60.90
4,2017-01-01,2451,18,25,588.24


In [92]:
os.listdir(transpath + 'cut-3-daily-spend-by-brand/'), os.listdir(transpath + 'cut-3-daily-spend-by-brand/2020-04-14')

(['2020-04-14', '2020-04-22', '2020-05-19'],
 ['daily-spend-by-brand-20170101-20200405-1.csv.gz',
  'daily-spend-by-brand-20170101-20200405-2.csv.gz',
  'daily-spend-by-brand-20170101-20200405-3.csv.gz',
  'daily-spend-by-brand-20170101-20200405-4.csv.gz'])

In [94]:
d = pd.read_csv(transpath + 'cut-3-daily-spend-by-brand/2020-04-14/daily-spend-by-brand-20170101-20200405-1.csv.gz', compression = 'gzip', header = None)
d.columns = ['date', 'merchant_name', 'sg_brandid', 'cardholder_zip', 'num_cards', 'num_trans', 'total_spend']
d.head()

,date,merchant_name,sg_brandid,cardholder_zip,num_cards,num_trans,total_spend
0,2017-01-01,24 HOUR FITNESS,SG_BRAND_1dff6738894adf75487b2c1ba5f54873,80128,1,1,36.00
1,2017-01-01,24 HOUR FITNESS,SG_BRAND_1dff6738894adf75487b2c1ba5f54873,80221,1,1,30.44
2,2017-01-01,7-ELEVEN,SG_BRAND_2e2448f12fc87fe452e9166153797a7f,6451,3,3,68.11
3,2017-01-01,7-ELEVEN,SG_BRAND_2e2448f12fc87fe452e9166153797a7f,6706,10,10,80.98
4,2017-01-01,7-ELEVEN,SG_BRAND_2e2448f12fc87fe452e9166153797a7f,7111,17,21,204.44


In [98]:
os.listdir(transpath+'cut-4-daily-spend-at-walmart/'), os.listdir(transpath+'cut-4-daily-spend-at-walmart/2020-04-22/')

(['2020-04-22'],
 ['daily-spend-by-zip-walmart-instore-20170101-20200417.csv.gz',
  'daily-spend-by-zip-walmart-online-20170101-20200417.csv.gz'])

In [101]:
d = pd.read_csv(transpath + 'cut-4-daily-spend-at-walmart/2020-04-22/daily-spend-by-zip-walmart-instore-20170101-20200417.csv.gz', compression = 'gzip', header = None, sep = '|')
d.columns = ['date', 'cardholder_zip', 'num_cards', 'num_trans', 'total_spend'] # in store
d.head()

,date,cardholder_zip,num_cards,num_trans,total_spend
0,2017-01-01,612,2,2,86.24
1,2017-01-01,617,1,1,3.88
2,2017-01-01,646,2,2,19.23
3,2017-01-01,650,1,1,1.00
4,2017-01-01,680,1,1,4.12


In [102]:
d = pd.read_csv(transpath + 'cut-4-daily-spend-at-walmart/2020-04-22/daily-spend-by-zip-walmart-online-20170101-20200417.csv.gz', compression = 'gzip', header = None, sep = '|')
d.columns = ['date', 'cardholder_zip', 'num_cards', 'num_trans', 'total_spend'] # online
d.head()

,date,cardholder_zip,num_cards,num_trans,total_spend
0,2017-01-01,1040,2,2,118.73
1,2017-01-01,1108,1,1,9.77
2,2017-01-01,1420,1,1,19.21
3,2017-01-01,1588,1,1,177.94
4,2017-01-01,1840,1,1,188.48


In [104]:
os.listdir(transpath+'cut-5-daily-federal-deposits-by-zip/'), os.listdir(transpath+'cut-5-daily-federal-deposits-by-zip/2020-04-28/')

(['2020-04-28'],
 ['cut-5-daily-federal-deposit-by-zip-20170101-20200417.csv.gz'])

In [105]:
d = pd.read_csv(transpath+'cut-5-daily-federal-deposits-by-zip/2020-04-28/'+'cut-5-daily-federal-deposit-by-zip-20170101-20200417.csv.gz')
d.head() # number of transactions and amount in zip code by no. of cards

,day,zip,num_cards,num_transaction,total_amount
0,2017-01-01,1550,1,1,7906.97
1,2017-01-01,2121,1,1,6778.54
2,2017-01-01,6051,1,1,7073.45
3,2017-01-01,7018,1,1,2555.12
4,2017-01-01,10566,1,1,189.07


# Core places

In [108]:
placepath = 'D://safegraph/coreplaces/'
os.listdir(placepath)

['brand_info.csv',
 'core_poi-part1.csv.gz',
 'core_poi-part2.csv.gz',
 'core_poi-part3.csv.gz',
 'core_poi-part4.csv.gz',
 'core_poi-part5.csv.gz',
 'README.txt']

In [109]:
d = pd.read_csv(placepath + 'core_poi-part1.csv.gz', compression = 'gzip')

In [110]:
d.head()

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags
0,sg:001341fe7e794ab6bd65bb80759a1ac6,NaN,Old Firehouse School,NaN,NaN,Child Day Care Services,Child Day Care Services,624410.0,37.890739,-122.118320,984 Moraga Rd,Lafayette,CA,94549,US,1.925284e+10,"{ ""Mon"": [[""8:00"", ""17:30""]], ""Tue"": [[""8:00"",...",NaN
1,sg:002921847f104c93a6df47bb831dedd2,NaN,Tenth Avenue Park,NaN,NaN,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,712190.0,42.255103,-89.085241,825 10th St,Rockford,IL,61104,US,NaN,NaN,NaN
2,sg:002d2e9f047d47ca8661abb8939096cf,sg:ee6502a0198b432091808af9c2f69e28,Hunter Village Wines & Liquors,NaN,NaN,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",445310.0,42.214633,-74.220827,7261 Rt 23a,Hunter,NY,12442,US,1.518263e+10,"{ ""Mon"": [], ""Tue"": [[""13:00"", ""18:00""]], ""Wed...",NaN
3,sg:0060616e5f184585a026a0b7e8192fbd,NaN,Fat Moe's,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511.0,40.948824,-74.154553,204 Diamond Bridge Ave,Hawthorne,NJ,7506,US,1.973949e+10,NaN,"Mediterranean Food,Smoothie & Juice Bar,Sandwi..."
4,sg:0061c1d8e124406cbcc97e40eb1dd585,NaN,Jungle Rapids Family Fun Park,NaN,NaN,Amusement Parks and Arcades,Amusement and Theme Parks,713110.0,34.207249,-77.863010,5320 Oleander Dr,Wilmington,NC,28403,US,1.910791e+10,NaN,NaN


In [112]:
d = pd.read_csv(placepath + 'brand_info.csv')
d.head()

,safegraph_brand_id,brand_name,parent_safegraph_brand_id,naics_code,top_category,sub_category,stock_symbol,stock_exchange
0,SG_BRAND_12b20c8402b220acfe0e65dd113dbbc7,&pizza,NaN,722511,Restaurants and Other Eating Places,Full-Service Restaurants,NaN,NaN
1,SG_BRAND_6462594f4a598a94,1 Hotels,NaN,721110,Traveler Accommodation,Hotels (except Casino Hotels) and Motels,NaN,NaN
2,SG_BRAND_407913c3eaadbdf961cbb276649f6618,1-800-Flowers,NaN,453110,Florists,Florists,FLWS,NASDAQ
3,SG_BRAND_98d0a05f33b79c60dffb8f23a236748f,10 Fitness,NaN,713940,Other Amusement and Recreation Industries,Fitness and Recreational Sports Centers,NaN,NaN
4,SG_BRAND_1d114312ec5fbd09,10 Gym,NaN,713940,Other Amusement and Recreation Industries,Fitness and Recreational Sports Centers,NaN,NaN


# Core places supplement

In [114]:
placesuppath = 'D://safegraph/'
d = pd.read_csv(placesuppath+'corepoi_sup.csv.gz')
d.head() # sqft of venue

,safegraph_place_id,location_name,polygon_class,is_synthetic,includes_parking_lot,iso_country_code,area_square_feet
0,sg:000190647c574858930fbf286b4e8ab8,Liberty Tax Service,SHARED_POLYGON,False,False,CA,38193
1,sg:001341fe7e794ab6bd65bb80759a1ac6,Old Firehouse School,OWNED_POLYGON,False,False,US,5495
2,sg:002921847f104c93a6df47bb831dedd2,Tenth Avenue Park,OWNED_POLYGON,False,NaN,US,388046
3,sg:002d2e9f047d47ca8661abb8939096cf,Hunter Village Wines & Liquors,SHARED_POLYGON,False,False,US,687
4,sg:0060616e5f184585a026a0b7e8192fbd,Fat Moe's,SHARED_POLYGON,False,False,US,10925


# Open census

In [119]:
censuspath = 'D://safegraph/open_census/safegraph_open_census_data/'
os.listdir(censuspath),os.listdir(censuspath+'data/'), os.listdir(censuspath+'geometry/') 

(['.DS_Store',
  '._.DS_Store',
  '._data',
  '._geometry',
  '._metadata',
  'data',
  'geometry',
  'metadata'],
 ['._cbg_b00.csv',
  '._cbg_b01.csv',
  '._cbg_b02.csv',
  '._cbg_b03.csv',
  '._cbg_b07.csv',
  '._cbg_b08.csv',
  '._cbg_b09.csv',
  '._cbg_b11.csv',
  '._cbg_b12.csv',
  '._cbg_b14.csv',
  '._cbg_b15.csv',
  '._cbg_b19.csv',
  '._cbg_b20.csv',
  '._cbg_b21.csv',
  '._cbg_b22.csv',
  '._cbg_b23.csv',
  '._cbg_b25.csv',
  '._cbg_b27.csv',
  '._cbg_b99.csv',
  '._cbg_c16.csv',
  '._cbg_c17.csv',
  '._cbg_c24.csv',
  '._cbg_patterns.csv',
  'cbg_b00.csv',
  'cbg_b01.csv',
  'cbg_b02.csv',
  'cbg_b03.csv',
  'cbg_b07.csv',
  'cbg_b08.csv',
  'cbg_b09.csv',
  'cbg_b11.csv',
  'cbg_b12.csv',
  'cbg_b14.csv',
  'cbg_b15.csv',
  'cbg_b19.csv',
  'cbg_b20.csv',
  'cbg_b21.csv',
  'cbg_b22.csv',
  'cbg_b23.csv',
  'cbg_b25.csv',
  'cbg_b27.csv',
  'cbg_b99.csv',
  'cbg_c16.csv',
  'cbg_c17.csv',
  'cbg_c24.csv',
  'cbg_patterns.csv'],
 ['.DS_Store', '._.DS_Store', '._cbg.geojson',

In [122]:
d = pd.read_csv(censuspath + 'data/cbg_c24.csv')
d.head()

,census_block_group,C24010e1,C24010m1,C24010e2,C24010m2,C24010e3,C24010m3,C24010e4,C24010m4,C24010e5,...,B24080e17,B24080m17,B24080e18,B24080m18,B24080e19,B24080m19,B24080e20,B24080m20,B24080e21,B24080m21
0,10010201001,301,96,139,56,38,26,28,28,23,...,6,9,0,11,6,9,16,24,0,11
1,10010201002,683,144,376,99,86,45,52,42,46,...,14,23,32,17,24,16,5,7,0,11
2,10010202001,330,148,189,104,44,36,28,35,28,...,0,11,39,39,0,11,0,11,0,11
3,10010202002,527,135,233,63,84,47,34,28,15,...,6,10,29,28,0,11,50,69,0,11
4,10010203001,1115,199,544,128,88,50,43,35,32,...,28,26,44,29,30,36,73,68,0,11


In [123]:
d = pd.read_csv(censuspath + 'data/cbg_patterns.csv') # summary of visits
d.head()

,census_block_group,date_range_start,date_range_end,raw_visit_count,raw_visitor_count,visitor_home_cbgs,visitor_work_cbgs,distance_from_home,related_same_day_brand,related_same_month_brand,top_brands,popularity_by_hour,popularity_by_day
0,1.005951e+10,1538352000,1541030400,75122.0,18314.0,"{""010059501003"":127,""010059509001"":111,""010059...","{""010059501003"":109,""010810407002"":62,""0108104...",194724.0,"[""Chick-fil-A"",""mcdonalds"",""Marathon Petroleum...","[""walmart"",""mcdonalds"",""Dollar General"",""Chick...","[""CrossFit"",""Health Mart"",""Coldwell Banker""]","[2617,2457,2403,2519,2646,3007,3886,7566,5508,...","{""Monday"":12000,""Tuesday"":12224,""Wednesday"":10..."
1,1.009051e+10,1538352000,1541030400,95649.0,38942.0,"{""010730113021"":210,""010090506022"":205,""010090...","{""010890111001"":271,""010730045001"":269,""010439...",120587.0,"[""Shell Oil"",""mcdonalds"",""Chick-fil-A"",""Chevron""]","[""walmart"",""mcdonalds"",""Shell Oil"",""Chick-fil-...",[],"[6556,6325,6222,6355,6586,7350,8568,8099,7378,...","{""Monday"":12125,""Tuesday"":12984,""Wednesday"":12..."
2,1.047957e+10,1538352000,1541030400,14009.0,3039.0,"{""010479567011"":67,""010479567021"":60}","{""010479567021"":52}",67774.0,"[""Dollar General""]","[""walmart"",""Dollar General"",""mcdonalds"",""Chevr...","[""Dollar General""]","[807,790,796,786,851,951,1134,1797,1355,1241,1...","{""Monday"":2314,""Tuesday"":2340,""Wednesday"":2195..."
3,1.069040e+10,1538352000,1541030400,128169.0,25418.0,"{""010690402013"":370,""010690402011"":322,""010690...","{""010690402024"":313,""010690415004"":203,""010450...",42684.0,"[""Chick-fil-A"",""Sam's Club"",""Dollar General"",""...","[""walmart"",""Dollar General"",""mcdonalds"",""Marat...","[""Chick-fil-A"",""Sam's Club"",""Olive Garden"",""mc...","[2121,1828,1784,1704,1861,2373,3730,7497,7093,...","{""Monday"":21141,""Tuesday"":21143,""Wednesday"":17..."
4,1.073011e+10,1538352000,1541030400,51453.0,9499.0,"{""010090507001"":183,""010730113021"":167,""010730...","{""010730045001"":140,""010730027001"":123,""010730...",18878.0,"[""Chevron"",""Daylight Donuts"",""walmart""]","[""walmart"",""Chevron"",""Dollar General"",""Shell O...","[""Chevron"",""CrossFit""]","[3804,3716,3686,3672,3735,4115,4855,5946,4526,...","{""Monday"":8402,""Tuesday"":8414,""Wednesday"":8550..."


In [134]:
d = pd.read_csv(censuspath + 'metadata/cbg_field_descriptions.csv')
d.head()

,table_id,field_full_name,field_level_1,field_level_2,field_level_3,field_level_4,field_level_5,field_level_6,field_level_7,field_level_8
0,B00001e1,UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Tot...,Unweighted Sample Count Of The Population,Total,Total Population -- (Estimate),NaN,NaN,NaN,NaN,NaN
1,B00001m1,UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Tot...,Unweighted Sample Count Of The Population,Total,Total Population -- (Margin Of Error),NaN,NaN,NaN,NaN,NaN
2,B00002e1,UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housin...,Unweighted Sample Housing Units,Total,Housing Units -- (Estimate),NaN,NaN,NaN,NaN,NaN
3,B00002m1,UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housin...,Unweighted Sample Housing Units,Total,Housing Units -- (Margin Of Error),NaN,NaN,NaN,NaN,NaN
4,B01001e1,SEX BY AGE: Total: Total population -- (Estimate),Sex By Age,Total,Total Population -- (Estimate),NaN,NaN,NaN,NaN,NaN


In [136]:
d = pd.read_csv(censuspath + 'metadata/cbg_fips_codes.csv')
d.head()

,state,state_fips,county_fips,county,class_code
0,AL,1,1,Autauga County,H1
1,AL,1,3,Baldwin County,H1
2,AL,1,5,Barbour County,H1
3,AL,1,7,Bibb County,H1
4,AL,1,9,Blount County,H1


In [137]:
d = pd.read_csv(censuspath + 'metadata/cbg_geographic_data.csv')
d.head()

,census_block_group,amount_land,amount_water,latitude,longitude
0,10010201001,4254524.0,23431,32.465829,-86.489614
1,10010201002,5568295.0,0,32.485849,-86.489690
2,10010202001,2058380.0,0,32.480082,-86.474974
3,10010202002,1283506.0,5968,32.464435,-86.469766
4,10010203001,3866515.0,9054,32.480175,-86.460793


In [125]:
if 1==0:
    d = geopandas.read_file(censuspath + 'geometry/cbg.geojson', engine = 'GEOJSON')
    d.head()

,StateFIPS,CountyFIPS,TractCode,BlockGroup,CensusBlockGroup,State,County,ClassCode,geometry
0,01,081,041600,1,010810416001,AL,Lee County,H1,"MULTIPOLYGON (((-85.37282 32.63424, -85.37275 ..."
1,01,081,041600,2,010810416002,AL,Lee County,H1,"MULTIPOLYGON (((-85.38346 32.64838, -85.38301 ..."
2,01,081,041700,4,010810417004,AL,Lee County,H1,"MULTIPOLYGON (((-85.37139 32.60139, -85.37138 ..."
3,01,073,011107,4,010730111074,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.64797 33.59205, -86.64771 ..."
4,01,073,011108,4,010730111084,AL,Jefferson County,H1,"MULTIPOLYGON (((-86.65206 33.59869, -86.65204 ..."


In [133]:
if 1==0: # Save smaller file
    tolerance = .01/69 # 1 degree is about 69 miles at equator
    d.geometry = d.geometry.simplify(tolerance, preserve_topology=True)
    fpath = '../../../Data/US/working_datasets/'
    d.to_file(fpath+'cbg_reduce_2019.geojson', driver='GeoJSON')